In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import math

loansData = pd.read_csv('https://spark-public.s3.amazonaws.com/dataanalysis/loansData.csv')
loansData.dropna(inplace=True)

In [3]:
print loansData.head()

       Amount.Requested  Amount.Funded.By.Investors Interest.Rate Loan.Length  \
81174             20000                       20000         8.90%   36 months   
99592             19200                       19200        12.12%   36 months   
80059             35000                       35000        21.98%   60 months   
15825             10000                        9975         9.99%   36 months   
33182             12000                       12000        11.71%   36 months   

             Loan.Purpose Debt.To.Income.Ratio State Home.Ownership  \
81174  debt_consolidation               14.90%    SC       MORTGAGE   
99592  debt_consolidation               28.36%    TX       MORTGAGE   
80059  debt_consolidation               23.81%    CA       MORTGAGE   
15825  debt_consolidation               14.30%    KS       MORTGAGE   
33182         credit_card               18.78%    NJ           RENT   

       Monthly.Income FICO.Range  Open.CREDIT.Lines  Revolving.CREDIT.Balance  \
81174

In [4]:
cleanInterestRate = loansData['Interest.Rate'].map(lambda x: round(float(x.rstrip('%')) / 100, 4))
type(cleanInterestRate)
loansData['Interest.Rate']= cleanInterestRate

In [5]:
cleanFICORange = loansData['FICO.Range'].map(lambda x: int(str(x).split('-')[0]))
type(cleanFICORange)
loansData['FICO.Range']= cleanFICORange

In [6]:
cleanAmountRequested = loansData['Loan.Length'].map(lambda x: int( str(x.rstrip(' months')) ))
type(cleanAmountRequested)
loansData['Loan.Length']= cleanAmountRequested

In [7]:
loansData['IR_TF'] = loansData['Interest.Rate'].map(lambda x: 0 if (x<0.12) else 1)

In [8]:
loansData['intercept'] = 1
ind_vars = ['intercept','Amount.Requested', 'FICO.Range']

logit = sm.Logit(loansData['IR_TF'], loansData[ind_vars])

result = logit.fit()
coeff = result.params
print coeff

Optimization terminated successfully.
         Current function value: 0.319276
         Iterations 8
intercept           60.171952
Amount.Requested     0.000174
FICO.Range          -0.087480
dtype: float64


In [9]:
def logistic_function( FicoScore,LoanAmount):
    p = 1/(1 + math.exp( coeff[0] + (coeff[1]*LoanAmount) + (coeff[2]*FicoScore) ) )
    return p

In [10]:
print logistic_function(720,10000)

0.745904602215


In [16]:
for index,row in loansData.iterrows():
    loansData.loc[index,'logistic_5'] = logistic_function(row['FICO.Range'],row['Amount.Requested'])
    #print row['IR_TF'], logistic_function(row['FICO.Range'],row['Amount.Requested'])
#print loansData.head()   

print loansData.groupby(['IR_TF']).mean()

       Amount.Requested  Amount.Funded.By.Investors  Interest.Rate  \
IR_TF                                                                
0          10503.888889                10097.047758       0.089759   
1          13666.379310                13264.659025       0.157584   

       Loan.Length  Monthly.Income  FICO.Range  Open.CREDIT.Lines  \
IR_TF                                                               
0        38.327273     5821.118343   734.79798           9.775758   
1        43.193634     5595.976267   686.86008          10.272546   

       Revolving.CREDIT.Balance  Inquiries.in.the.Last.6.Months  intercept  \
IR_TF                                                                        
0                  14932.715152                        0.694949          1   
1                  15449.285146                        1.045093          1   

       logistic_5  
IR_TF              
0        0.751871  
1        0.162897  


In [57]:
temp = loansData

#temp['logistic1'] = temp['IR_TF'].map(lambda x: logistic_function(x['FICO.Range'],x['Amount.Requested']))
#print temp

for index,row in loansData.iterrows():
    temp['logistic2'] = logistic_function(row['FICO.Range'],row['Amount.Requested'])
    
print temp.head()

       Amount.Requested  Amount.Funded.By.Investors  Interest.Rate  \
81174             20000                       20000         0.0890   
99592             19200                       19200         0.1212   
80059             35000                       35000         0.2198   
15825             10000                        9975         0.0999   
33182             12000                       12000         0.1171   

       Loan.Length        Loan.Purpose Debt.To.Income.Ratio State  \
81174           36  debt_consolidation               14.90%    SC   
99592           36  debt_consolidation               28.36%    TX   
80059           60  debt_consolidation               23.81%    CA   
15825           36  debt_consolidation               14.30%    KS   
33182           36         credit_card               18.78%    NJ   

      Home.Ownership  Monthly.Income  FICO.Range  Open.CREDIT.Lines  \
81174       MORTGAGE         6541.67         735                 14   
99592       MORTGAGE  

In [44]:
temp1 = temp.groupby('IR_TF')
print temp1.mean()

       Amount.Requested  Amount.Funded.By.Investors  Interest.Rate  \
IR_TF                                                                
0          10503.888889                10097.047758       0.089759   
1          13666.379310                13264.659025       0.157584   

       Loan.Length  Monthly.Income  FICO.Range  Open.CREDIT.Lines  \
IR_TF                                                               
0        38.327273     5821.118343   734.79798           9.775758   
1        43.193634     5595.976267   686.86008          10.272546   

       Revolving.CREDIT.Balance  Inquiries.in.the.Last.6.Months  intercept  \
IR_TF                                                                        
0                  14932.715152                        0.694949          1   
1                  15449.285146                        1.045093          1   

       logistic  logistic1  
IR_TF                       
0      0.042151   0.042151  
1      0.042151   0.042151  
